In [1]:
import pandas as pd
import os
import io
import datetime

In [2]:
start_time = '2014-04-01'
# 1. Convert the_day string to a datetime object
date_obj = datetime.datetime.strptime(start_time, '%Y-%m-%d')
end_time = date_obj + datetime.timedelta(days=6)
print(end_time)
#end_time = '2014-04-07'
one_week_data = pd.read_csv('one_week_data.csv')
one_week_data['Timestamp'] = pd.to_datetime(one_week_data['Timestamp'])
one_week_data.head(5)

2014-04-07 00:00:00


,Unnamed: 0,SensorID,Attribute1,Attribute2,Value,SensorType,Timestamp
0,6431809,BATP105,Ignore,Ignore,36,Control4-BatteryPercent,2014-04-01 02:43:32.162533
1,6431810,BATP103,Ignore,Ignore,48,Control4-BatteryPercent,2014-04-01 03:21:14.529900
2,6431811,T105,Ignore,KitchenTemp,25,Control4-Temperature,2014-04-01 04:12:48.248098
3,6431812,BATP022,Ignore,Ignore,42,Control4-BatteryPercent,2014-04-01 05:09:01.748707
4,6431813,BATP104,Ignore,Ignore,45,Control4-BatteryPercent,2014-04-01 05:48:18.736504


In [3]:
the_day = '2014-04-05'

# 1. Convert the_day string to a datetime object
date_obj = datetime.datetime.strptime(the_day, '%Y-%m-%d')

# 2. Calculate 9 PM of the given date
nine_pm_current_day = date_obj.replace(hour=21, minute=0, second=0, microsecond=0)

# 3. Calculate 9 PM of one day earlier
one_day_earlier = date_obj - datetime.timedelta(days=1)
nine_pm_one_day_earlier = one_day_earlier.replace(hour=21, minute=0, second=0, microsecond=0)

# 4. Get the timestamps (Unix timestamps)
timestamp_nine_pm_one_day_earlier = nine_pm_one_day_earlier.timestamp()
timestamp_nine_pm_current_day = nine_pm_current_day.timestamp()

print(f"Date used: {the_day}")
print(f"9 PM one day earlier ({nine_pm_one_day_earlier}): {timestamp_nine_pm_one_day_earlier}")
print(f"9 PM of the date ({nine_pm_current_day}): {timestamp_nine_pm_current_day}")


Date used: 2014-04-05
9 PM one day earlier (2014-04-04 21:00:00): 1396616400.0
9 PM of the date (2014-04-05 21:00:00): 1396702800.0


In [5]:
one_day_data = one_week_data[(one_week_data['Timestamp'] >= nine_pm_one_day_earlier) 
    & (one_week_data['Timestamp'] <= nine_pm_current_day)]
one_day_data.head(5)

,Unnamed: 0,SensorID,Attribute1,Attribute2,Value,SensorType,Timestamp
2729,6434538,LS008,Ignore,Ignore,24,Control4-LightSensor,2014-04-04 21:00:36.206586
2730,6434539,BATP008,Ignore,Ignore,63,Control4-BatteryPercent,2014-04-04 21:00:36.258214
2731,6434540,LS007,Ignore,Ignore,8,Control4-LightSensor,2014-04-04 21:00:44.880928
2732,6434541,M007,Kitchen,Kitchen,ON,Control4-Motion,2014-04-04 21:00:44.941567
2733,6434542,LS007,Ignore,Ignore,9,Control4-LightSensor,2014-04-04 21:00:46.945409


In [16]:
# 1. Filter for Motion Sensors
motion_df = one_day_data[(one_day_data['SensorType'] == 'Control4-Motion') & (one_day_data['Value'] == 'ON')].copy()

# 2. Sort by Timestamp
motion_df = motion_df.sort_values(by='Timestamp').reset_index(drop=True)

# 3. Simplify 'Value' to boolean for easier processing
motion_df['IsMotion'] = motion_df['Value'] == 'ON'
# 4. create a new column end time
motion_df['end_time'] = motion_df['Timestamp']


In [21]:
# Assign group IDs when the room name changes
motion_df['GroupID'] = (motion_df['Attribute1'] != motion_df['Attribute1'].shift()).cumsum()
# motion_df.head(20)
# Group by GroupID and extract the first start time and last end time for each group
result_df = stay_periods_df.groupby('GroupID').agg({
    'Room': 'first',
    'Start Time': 'first',
    'End Time': 'last'
}).reset_index(drop=True)

result_df.head(10)

,Room,Start Time,End Time
0,Kitchen,2014-04-04 21:00:44.941567,2014-04-04 21:00:58.212312
1,DiningRoom,2014-04-04 21:00:58.212312,2014-04-04 21:01:00.626681
2,LivingRoom,2014-04-04 21:04:51.069809,2014-04-04 21:09:51.214350
3,DiningRoom,2014-04-04 21:09:55.781471,2014-04-04 21:10:08.622916
4,WorkArea,2014-04-04 21:10:41.732679,2014-04-04 21:10:42.850097
5,DiningRoom,2014-04-04 21:11:20.800523,2014-04-04 21:11:51.780999
6,Kitchen,2014-04-04 21:12:18.611104,2014-04-04 21:12:19.333686
7,DiningRoom,2014-04-04 21:12:19.333686,2014-04-04 21:12:21.017878
8,LivingRoom,2014-04-04 21:13:02.273181,2014-04-04 21:13:51.168442
9,DiningRoom,2014-04-04 21:14:56.440836,2014-04-04 21:14:58.120701
